[View in Colaboratory](https://colab.research.google.com/github/stoiver/anuga-clinic-2018/blob/master/notebooks/notebook3.ipynb)

# CSDMS ANUGA Clinic 2018

## Notebook 3: Create an Erosion Operator 

Here we go through the process of creating an operator (fractional step operator) which will implement a simple sand dune erosion operator. 

These notebooks have been designed to run in the google `colaboratory` environment, which provides a jupyter notebook environment running on a virtual machine on the cloud. To use this environment you need a google account so that your notebook can be saved on google drive. 

To start interacting with the notebook follow the 
`View in Colaboratory` link above. 

## Setup Environment

If on github, first follow the link `View in Colaboratory' to start running on google's colab environment. Then ....

Run the following cell to install the dependencies and some extra code for visualising on Colaboratory.

Wait until you see the comment *(5) Ready to go* before proceeding to subsequent commands. 

The install should take less than a minute (and quicker if you have already run this earlier).

In [1]:
# First download the clinic repository
import os
os.chdir('/content')
!git clone https://github.com/stoiver/anuga-clinic-2018.git

# Now install environment using tool
!/bin/bash /content/anuga-clinic-2018/anuga_tools/install_anuga_colab.sh

# Make inline animate code available
import sys
sys.path.append("/content/anuga-clinic-2018")


import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

import anuga
import anuga_tools.animate as animate

Cloning into 'anuga-clinic-2018'...
remote: Counting objects: 403, done.
remote: Total 403 (delta 0), reused 0 (delta 0), pack-reused 403
Receiving objects: 100% (403/403), 2.28 MiB | 21.83 MiB/s, done.
Resolving deltas: 100% (158/158), done.
(1) Install netcdf nose via pip
(2) Install gdal via apt-get
(3) Download anuga_core github repository
(4) Install anuga
(5) Ready to go


## The Code 

In [62]:
"""sand dune erosion test                          run_sanddune_erosion.py

This tests the sanddune_erosion operator confirming that; 
    1. flow creates erosion when bed shear > critical and that erosion rates 
	are higher in higher bed shear zones (typically higher velocity areas).
	2. that erosion is augmented by collapse of the sand face whenever
	erosion creates slopes > the angle of repose.  
	this process leads to widening of the notch (laterally)and head
    like recession of 
	the main scour zone  as scour acts to steepen the longitudinal 
	bed, triggering collapse of the steep bed and reshaping back 
	to the angle of repose).
	3. that the operator can handle multiple erosion polygons with
    different base levels
	 
"""

#------------------------------------------------------------------------------
# Import necessary modules
#------------------------------------------------------------------------------
import anuga
from anuga import Region, Geo_reference
import numpy as num

x0 = 314036.58727982
y0 = 6224951.2960092
geo = Geo_reference(56, x0, y0)

#------------------------------------------------------------------------------
# Function to describe topography as function of X and y
#------------------------------------------------------------------------------
def topography(x,y):
    """Complex topography defined by a function of vectors x and y."""
    print ' Creating topography....'
    
    z = 0.0*(x)                             # horizontal plane 

    N = len(x)
   
    for i in range(N):
        # First notched sand dune across Channel
        if 1010.6 < x[i] <= 1012.0:
            z[i] +=  1.0*(x[i] -1010.6)      # Sloping U/S Face 1:1
        if 1012.0 < x[i] < 1013.0 :
            z[i] +=  1.4                   # Crest of Embankment at +1.4
        if 1013.0 <= x[i] < 1014.4:
            z[i] +=  1.4-1.0*(x[i] -1013.0)  # Sloping D/S Face at 1:1

        # add notch in crest 1m wide by nom 300 deep
        # note sides are near vertical so will collapse back to repose even without erosion

        if 1011.7 <= x[i] <= 1013.3 and 1002.0 <= y[i] <= 1003.0:
            z[i] =  1.1                   # add 300 Notch in Embankment crest  
        # second lower plain sand dune across Channel
        if 1023.0 < x[i] <= 1024.0:
            z[i] +=  1.0*(x[i] -1023.0)      # Sloping U/S Face 1:1        
        if 1024.0 < x[i] < 1025.0 :
            z[i] +=  1.0                   # Crest of Embankment at +1.0
        if 1025.0 <= x[i] < 1026.0:
            z[i] +=  1.0-1.0*(x[i] -1025.0)  # Sloping D/S Face at 1:1      
    return z

#------------------------------------------------------------------------------
# build the check points and erosion polygons now so available to all processors
#------------------------------------------------------------------------------

polygon1    = num.array([ [1010.6, 1000.0], [1014.4, 1000.0], [1014.4, 1005.0], [1010.6, 1005.0] ])
polygon2    = num.array([ [1023.0, 1000.0], [1026.0, 1000.0], [1026.0, 1005.0], [1023.0, 1005.0] ])
poly1nsbase = 0.5  # set poly no scour base level in m
poly2nsbase = 0.3

polygon1 += num.array([x0,y0])
polygon2 += num.array([x0,y0])

#------------------------------------------------------------------------------
# Setup computational domain
#------------------------------------------------------------------------------

print '>>>>> DUNE EROSION TEST SCRIPT V2'
print '>>>>> Setting  up Domain on processor 0...'
length = 36.
width = 5.
dx = dy = 0.25            # Resolution: Length of subdivisions on both axes
print '>>>>> Domain has L = %f, W = %f, dx=dy= %f' %(length,width,dx) 
points, vertices, boundary = anuga.rectangular_cross(int(length/dx), int(width/dy), len1=length, len2=width)
points = points + 1000.0


domain = anuga.Domain(points, vertices, boundary, geo_reference = geo)
domain.set_flow_algorithm('DE0')
domain.set_name('sanddune') # Output name

domain.set_store_vertices_uniquely(False)
domain.set_quantities_to_be_stored({'elevation': 2,'stage': 2,'xmomentum': 2,'ymomentum': 2})

domain.set_quantity('elevation', topography)           # elevation is a function
domain.set_quantity('friction', 0.01)                  # Constant friction
domain.set_quantity('stage', expression='elevation')   # Dry initial condition

print domain.statistics()

# get the indices of triangles in each erosion poly so can setup nsbase_ in domain
poly1ind = (Region(domain, polygon=polygon1)).indices
poly2ind = (Region(domain, polygon=polygon2)).indices
print '>>>>> poly1ind is of length ', len(poly1ind), ' and contains triangles', poly1ind[0], ' to ' , poly1ind[-1]
print '>>>>> poly2ind is of length ', len(poly2ind), ' and contains triangles', poly2ind[0], ' to ' , poly2ind[-1]

# get the initial model surface elevation
nsbase_elev_c = domain.get_quantity('elevation').get_values(location='centroids')
print '>>>>> nsbase_elev_c is of length ',len(nsbase_elev_c) 


# build the no scour base surface  by combining initial elev where < nsbase and nsbase in each scour poly
nsbase_elev_c[poly1ind] = num.minimum(nsbase_elev_c[poly1ind], poly1nsbase)
nsbase_elev_c[poly2ind] = num.minimum(nsbase_elev_c[poly2ind], poly2nsbase)


# create new Anuga quantity and assign nsbase_elev_c values to domain so can distribute later
anuga.Quantity(domain, name='nsbase_elevation', register=True)
domain.set_quantity('nsbase_elevation',nsbase_elev_c, location='centroids')




#------------------------------------------------------------------------------
# Print out polygon points and ids as check
#------------------------------------------------------------------------------
print '>>>>> erosion polygon1 contains ', polygon1
print '>>>>> erosion polygon2 contains ', polygon2



#------------------------------------------------------------------------------
# Setup boundary conditions on the distributed domain
#------------------------------------------------------------------------------
Bi = anuga.Dirichlet_boundary([1.2, 0, 0])          # Inflow at depth
Br = anuga.Reflective_boundary(domain)              # Solid reflective side walls
Bo = anuga.Dirichlet_boundary([-5, 0, 0])           # uncontrolled outflow 

domain.set_boundary({'left': Bi, 'right': Bo, 'top': Br, 'bottom': Br})

#------------------------------------------------------------------------------
# Setup sanddune erosion operator 
#------------------------------------------------------------------------------

print '>>>>> Setting up Erosion Area(s) to test...'

# power up the erosion operator
from anuga import Sanddune_erosion_operator

# assign ns base elevations to partitioned domains
nsbase_elev_c = domain.get_quantity('nsbase_elevation').get_values(location='centroids')

poly1ind = (Region(domain, polygon=polygon1)).indices
poly2ind = (Region(domain, polygon=polygon2)).indices
indices_union = list(set(poly1ind) | set(poly2ind))

# setup and create operator within polys setting the scour base elevations for each poly
op0 = Sanddune_erosion_operator(domain, base=nsbase_elev_c, indices=indices_union, Ra=45)   # both dunes
#op1 = sanddune_erosion_operator(domain, base=nsbase_elev_c, polygon=polygon1)   # first notched dune
#op2 = sanddune_erosion_operator(domain, base=nsbase_elev_c, polygon=polygon2)   # second plain dune


#------------------------------------------------------------------------------
# Evolve simulation through time  
#------------------------------------------------------------------------------
for t in domain.evolve(yieldstep=1.0, duration=50.0):
    domain.print_timestepping_statistics()
    

#  run completed - tidy up 
print ' >>>>> Simulation completed successfully '

 


>>>>> DUNE EROSION TEST SCRIPT V2
>>>>> Setting  up Domain on processor 0...
>>>>> Domain has L = 36.000000, W = 5.000000, dx=dy= 0.250000
 Creating topography....
------------------------------------------------
Mesh statistics:
  Number of triangles = 11520
  Extent [m]:
    x in [1.00000e+03, 1.03600e+03]
    y in [1.00000e+03, 1.00500e+03]
  Areas [m^2]:
    A in [1.56250e-02, 1.56250e-02]
    number of distinct areas: 11520
    Histogram:
      [1.56250e-02, 1.56250e-02]: 11520
    Percentiles (10 percent):
      1152 triangles in [1.56250e-02, 1.56250e-02]
      1152 triangles in [1.56250e-02, 1.56250e-02]
      1152 triangles in [1.56250e-02, 1.56250e-02]
      1152 triangles in [1.56250e-02, 1.56250e-02]
      1152 triangles in [1.56250e-02, 1.56250e-02]
      1152 triangles in [1.56250e-02, 1.56250e-02]
      1152 triangles in [1.56250e-02, 1.56250e-02]
      1152 triangles in [1.56250e-02, 1.56250e-02]
      1152 triangles in [1.56250e-02, 1.56250e-02]
      1152 triangles in

Time = 34.0000, delta t in [0.00925048, 0.00962003], steps=107 (0s)
Time = 35.0000, delta t in [0.00923849, 0.00966255], steps=106 (0s)
Time = 36.0000, delta t in [0.00923373, 0.00970057], steps=106 (0s)
Time = 37.0000, delta t in [0.00922582, 0.00970219], steps=106 (0s)
Time = 38.0000, delta t in [0.00921287, 0.00971363], steps=106 (0s)
Time = 39.0000, delta t in [0.00918413, 0.00972061], steps=106 (0s)
Time = 40.0000, delta t in [0.00915382, 0.00973070], steps=106 (0s)
Time = 41.0000, delta t in [0.00912945, 0.00972911], steps=107 (0s)
Time = 42.0000, delta t in [0.00910459, 0.00973346], steps=107 (0s)
Time = 43.0000, delta t in [0.00908849, 0.00974192], steps=107 (0s)
Time = 44.0000, delta t in [0.00908420, 0.00974215], steps=107 (0s)
Time = 45.0000, delta t in [0.00909573, 0.00973576], steps=107 (0s)
Time = 46.0000, delta t in [0.00909659, 0.00974387], steps=107 (0s)
Time = 47.0000, delta t in [0.00907850, 0.00976349], steps=107 (0s)
Time = 48.0000, delta t in [0.00906650, 0.009756

In [64]:
# Create a wrapper for contents of sww file
swwfile = 'sanddune.sww'
splotter = animate.SWW_plotter(swwfile)

Figure files for each frame will be stored in _plot


In [66]:
splotter.triang.set_mask(None)

figsize=(10,6)
dpi = 160
plot_dir = '_plot'
name = splotter.name
  
for i,time in enumerate(splotter.time):
  print 'time ',time
  
  depth = splotter.depth[i,:]
  elev  = splotter.elev[i,:]
  
  fig = plt.figure(figsize=figsize, dpi=dpi)

  plt.title('Depth: Time {0:0>4}'.format(time))

  plt.tripcolor(splotter.triang, 
            facecolors = elev,
            cmap='viridis')

  plt.colorbar()
  
  if plot_dir is None:
    plt.savefig(name+'_depth_{0:0>10}.png'.format(int(time)))
  else:
    plt.savefig(os.path.join(plot_dir, name+'_depth_{0:0>10}.png'.format(int(time))))
  
  plt.close()



splotter.make_depth_animation()

time  0.0
time  1.0
time  2.0
time  3.0
time  4.0
time  5.0
time  6.0
time  7.0
time  8.0
time  9.0
time  10.0
time  11.0
time  12.0
time  13.0
time  14.0
time  15.0
time  16.0
time  17.0
time  18.0
time  19.0
time  20.0
time  21.0
time  22.0
time  23.0
time  24.0
time  25.0
time  26.0
time  27.0
time  28.0
time  29.0
time  30.0
time  31.0
time  32.0
time  33.0
time  34.0
time  35.0
time  36.0
time  37.0
time  38.0
time  39.0
time  40.0
time  41.0
time  42.0
time  43.0
time  44.0
time  45.0
time  46.0
time  47.0
time  48.0
time  49.0
time  50.0


In [65]:
splotter.triang.set_mask(None)
for i,time in enumerate(splotter.time):
  print 'Time ',time
  splotter.save_depth_frame(i)
  
splotter.make_depth_animation()

Time  0.0
Time  1.0
Time  2.0
Time  3.0
Time  4.0
Time  5.0
Time  6.0
Time  7.0
Time  8.0
Time  9.0
Time  10.0
Time  11.0
Time  12.0
Time  13.0
Time  14.0
Time  15.0
Time  16.0
Time  17.0
Time  18.0
Time  19.0
Time  20.0
Time  21.0
Time  22.0
Time  23.0
Time  24.0
Time  25.0
Time  26.0
Time  27.0
Time  28.0
Time  29.0
Time  30.0
Time  31.0
Time  32.0
Time  33.0
Time  34.0
Time  35.0
Time  36.0
Time  37.0
Time  38.0
Time  39.0
Time  40.0
Time  41.0
Time  42.0
Time  43.0
Time  44.0
Time  45.0
Time  46.0
Time  47.0
Time  48.0
Time  49.0
Time  50.0


In [9]:
p2 = splotter.p2

In [10]:
p2.elev.shape

(72000,)

In [13]:
from anuga.file.netcdf import NetCDFFile as NetCDF

file = NetCDF(swwfile)

In [18]:
elevation = file.variables['elevation']
elevation.shape

(13, 36411)

In [20]:
elev_c  = file.variables['elevation_c']
stage_c = file.variables['stage_c']
elev_c.shape, stage_c.shape

((13, 72000), (13, 72000))

In [24]:
ls

anuga-clinic-2018/  anuga_core/  datalab/  _plot/  sanddune_testV2SR.sww


In [51]:
import os
os.chdir('anuga-clinic-2018')
!git pull
os.chdir('/content')
reload(animate)

remote: Counting objects: 4, done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/stoiver/anuga-clinic-2018
   98bbc8c..50d9d4c  master     -> origin/master
Updating 98bbc8c..50d9d4c
Fast-forward
 anuga_tools/animate.py | 10 ++++++++--
 1 file changed, 8 insertions(+), 2 deletions(-)


<module 'anuga_tools.animate' from '/content/anuga-clinic-2018/anuga_tools/animate.py'>

In [32]:
file

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    institution: Geoscience Australia
    description: Output from anuga.file.sww suitable for plotting
    smoothing: Yes
    vertices_are_stored_uniquely: False
    order: 2
    revision_number: 9737
    starttime: 0
    xllcorner: 314036.58727982
    yllcorner: 6224951.2960092
    zone: 56
    false_easting: 500000
    false_northing: 10000000
    datum: wgs84
    projection: UTM
    units: m
    dimensions(sizes): number_of_volumes(72000), number_of_triangle_vertices(36411), number_of_vertices(3), numbers_in_range(2), number_of_points(36411), number_of_timesteps(13)
    variables(dimensions): float32 x(number_of_points), float32 y(number_of_points), int32 volumes(number_of_volumes,number_of_vertices), float32 xmomentum(number_of_timesteps,number_of_points), float32 xmomentum_range(numbers_in_range), float32 elevation(number_of_timesteps,number_of_points), float32 elevation_range(num